# **Link to the Documentation:**
https://regal-cardinal-70d.notion.site/Report-on-AIS-Data-Preprocessing-and-Vessel-Trajectory-Analysis-Ahmed-Yehia-15298bf0dc9a8073801ae53a4103fa8b?pvs=4

# Imports

In [ ]:
%pip install tqdm
%pip install geopandas
%pip install folium
%pip install pyarrow
%pip install fastparquet
%pip install scikit-learn
%pip install statsmodels

In [ ]:
%pip freeze

In [1]:
import pandas as pd
import geopandas as gpd

In [ ]:
import importlib
import data_cleaning
import helper_utils
import helper_utils_level_1
import helper_utils_level_2
import helper_utils_level_3
import helper_utils_level_4

# Reading Data

In [ ]:
port_info = helper_utils.load_port_info()
port_polygons = helper_utils.load_port_polygons()
raw_data = helper_utils.load_raw_data("./data/raw_data/raw_data.parquet")

In [ ]:
port_info

In [ ]:
port_polygons

In [ ]:
raw_data

In [ ]:
len(raw_data["anonymized_mmsi"].unique())

In [ ]:
port_gdf = gpd.GeoDataFrame(port_polygons)
port_gdf.head()

# Section 1: Data Preprocessing

In [ ]:
importlib.reload(data_cleaning)
raw_data = data_cleaning.remove_NA(raw_data)
raw_data = data_cleaning.remove_invalid_lat_and_lon(raw_data)
raw_data = data_cleaning.remove_invalid_speed(raw_data)
raw_data = data_cleaning.convert_to_datetime(raw_data)

# Section 2: Analysis of Vessel Trajectories 

## Level 1: Direct Observations

### Detect Idle States:
- After researching online, I found that the recommended **threshold for speed** falls between **0.5 and 2 knots**.
- To allow flexibility, I implemented a function that takes this speed threshold as a **parameter**.

In [ ]:
importlib.reload(helper_utils_level_1)
idle_instances_data = helper_utils_level_1.detect_idle_states(data= raw_data, speed_threshold= 1)
idle_instances_data.head()

### Detect When a Vessel enters a port area:
The initial approach I implemented relied on a geo-spatial method. This approach checks whether a vessel's trajectory intersects with a given port’s polygon (i.e., whether the vessel is within the boundaries of the port).

In [ ]:
importlib.reload(helper_utils_level_1)
ais_gdf_output, visit_summary, map = helper_utils_level_1.visualize_vessel_trajectory_and_ports(
                                                    data= raw_data
                                                    ,port_gdf= port_polygons
                                                    ,anonymized_mmsi="dfcce95e07"
                                                    # ,map_save_directory="vessel_trajectory_with_ports.html"
                                                    )
map 

In [ ]:
visit_summary

In [ ]:
importlib.reload(helper_utils)
ais_gdf_output, visit_summary, map = helper_utils_level_1.visualize_vessel_trajectory_and_ports(
                                                    data= raw_data
                                                    ,port_gdf= port_polygons
                                                    ,anonymized_mmsi="f7ca11eb18"
                                                    ,map_save_directory="vessel_trajectory_with_ports.html"
                                                    )
map

In [ ]:
visit_summary

## Level 2: Derived Behaviours 

### Waiting Near Ports:

To identify vessels that may be waiting at a port, I implemented a **waiting period detection mechanism** based on the **vessel's location and speed**. This helped to identify instances where a vessel remained stationary for a significant amount of time near a port, which could indicate they were either:

- Waiting in a port queue.
- Docked at a berth for unloading or loading operations.


**Buffer Distance for Port Detection:**

In addition to the basic location checks, I introduced a **buffer distance** around the ports to account for vessels that might remain stationary outside the exact port area. Sometimes, vessels don't enter the port's exact polygon but still wait in close proximity, so adding a buffer allows for more **accurate detection** of such vessels.

- The **buffer distance** was set as **adjustable in degrees** to allow flexibility. Given that 1 degree of latitude/longitude is roughly equivalent to 111 kilometers at the Earth's surface, this adjustment allows for easy calibration depending on the geographical region or level of granularity needed.
- The added **buffer** ensures that vessels parked slightly outside the port’s boundary but still within a reasonable distance are also captured. This provides a more comprehensive analysis of vessel behaviors near ports.


**Logic for Waiting Periods:**

- **Vessel Location**: The vessel must be near a port (inside the buffer region).
- **Speed Condition**: The vessel must be stationary, which was determined by checking if the vessel's speed was below a specific threshold (e.g., 0.5 knots).
- **Time Threshold**: A minimum duration (e.g., 10 minutes) is required for the vessel to be considered as **waiting** in the area.

In [ ]:
importlib.reload(helper_utils_level_2)
waiting_events, map = helper_utils_level_2.detect_waiting_near_ports(
                                    data= raw_data
                                    ,port_gdf= port_polygons
                                    ,anonymized_mmsi="f7ca11eb18"
                                    ,buffer_distance= 0.2 # approximately 22.22 kilometers (1 degree = 111.111KM)
                                    ,min_wait_time= pd.Timedelta(minutes=30)
                                    ,max_speed= 2
                                    ,map_save_directory="vessel_waiting_near_prots.html"
                                    )
map

### Unserved Ports Visits:

Unserved port visits refer to instances where a vessel enters a port but either does not stay long enough or engages in suspicious behavior that suggests the visit was not fully serviced or completed. To identify these, I implemented a function that evaluates various conditions to mark a visit as unserved

**Logic for Identifying Unserved Visits:**

- **Entry Check**: The visit must have entered the port area (i.e., the vessel’s position is within the buffer distance of the port).
- **Duration Check**: If the duration of the visit is shorter than the specified minimum stay time (60 minutes), the visit is flagged as **unserved**.
- **Speed Check**: If the vessel’s maximum speed during the visit exceeds the threshold (`10 knots`), this indicates the vessel may not have stayed long enough or did not engage in port activities, marking the visit as **unserved**.
- **Speed Change Check**: If the vessel’s speed changed significantly (greater than `10 knots`), it suggests an abrupt movement, possibly indicating that the vessel left the port area quickly without completing its port services.

 Speed: Identifying vessels that approach ports but do not stop 
       (e.g., speed remains above a certain threshold). 
       
 Directionality: Vessels that enter the buffer zone but exit without significant directional changes 
                   or prolonged presence.

 Stationary Periods: Absence of significant movement (e.g., small changes in latitude/longitude) 
              while near a port.

 The speed changes abruptly within the visit.
 
 The vessel slows down but never stops completely (indicating potential unserved behavior).

In [ ]:
importlib.reload(helper_utils_level_2)
unserved_events, map = helper_utils_level_2.detect_unserved_visits_near_ports(
                                    data= raw_data
                                    ,port_gdf= port_polygons
                                    ,anonymized_mmsi="f7ca11eb18"
                                    ,buffer_distance= 0.2 # approximately 22.22 kilometers (1 degree = 111.111KM)
                                    ,max_speed_threshold=10 # if higher than thrsh then is unserved visit
                                    ,min_speed_change=10 # if speed change highrt then is unserved visit
                                    ,min_stay_time= pd.Timedelta(minutes=60)
                                    # ,map_save_directory="vessel_waiting_near_prots.html"
                                    )
map

In [ ]:
helper_utils.plot_one_trajectory(raw_data[raw_data["anonymized_mmsi"]=="f7ca11eb18"])

## Level 3:  Aggregated Patterns 

### Average Duration:

The average duration of vessel stays at specific locations.

The function first filters the dataset to focus on the region of interest using the provided `lat`, `lon`, and `buffer_radius`. Here's the breakdown of the implemented logic:

1. **Entry and Exit Identification:**
    - For each trajectory, entry and exit points within the defined buffer region are identified.
    - This ensures that only distinct visits to the location are analyzed.
2. **Average Duration Calculation:**
    - The duration for each distinct visit is computed based on the **difference between exit and entry timestamps**.
    - Average duration is then calculated for all visits. This helps provide insights into typical vessel behavior at the location.
3. **Special Cases Handled:**
    - If the average duration is **zero**, it indicates no stays occurred—only trajectories passed through the area.
    - If the result is **NaT (Not a Time)**, it means the vessel entered and exited the buffer **at the same time**, suggesting an instantaneous pass-through.

In [ ]:
importlib.reload(helper_utils_level_3)
avg_duration, map = helper_utils_level_3.calculate_average_duration_and_map(
                                            data= raw_data
                                            ,lat=46
                                            ,lon=-31 
                                            ,buffer_radius=1000 #in meters
                                            ,anonymized_mmsis=["dfcce95e07","deb97e3414"]
                                            # ,map_save_directory="vessel_waiting_near_prots.html"
                                            )
print(f"Average Duration of Stays: {avg_duration} - if zeros that means no stays, only trajectories - \n\
if NaT that means entered and exited the circle at the same time")
map

###  Peak Times:

Designed to analyze vessel activity across three temporal dimensions: hourly, daily, and monthly. This function provides valuable insights into traffic patterns, helping port authorities and logistics managers optimize operations.

**Customization:**
    - If specific parameters are provided (e.g., a port name or vessel MMSIs), the function tailors its analysis to those inputs.
    - When parameters are not passed, the analysis defaults to all ports and all vessels, ensuring flexibility.

**Key Observations from Charts**

1. **Hourly Analysis:**
    - Activity was **almost evenly distributed across all hours**, indicating no significant peaks or troughs in vessel operations on an hourly basis. This suggests consistent operations throughout the day.
2. **Daily Analysis:**
    - A **higher trend at the end of the week** was observed, possibly reflecting increased port activity during weekends or pre-weekend stocking/preparation.
3. **Monthly Analysis:**
    - Data availability was limited to **April and May**, indicating either seasonal traffic patterns or gaps in the dataset. This could reflect port-specific factors like seasonal trade, monsoon effects, or maintenance schedules.

In [ ]:
importlib.reload(helper_utils_level_3)
peak_times = helper_utils_level_3.identify_peak_times_at_port(
                                            data= raw_data
                                            ,port_gdf= port_polygons
                                            ,port_name= "Chennai"
                                            ,buffer_radius=1000 #in meters
                                            ,anonymized_mmsis=["dfcce95e07"]
                                            )
peak_times

In [ ]:
importlib.reload(helper_utils_level_3)
peak_times_hour, peak_times_month, peak_times_day_of_week = helper_utils_level_3.identify_peak_times_at_port(
                                            data= raw_data
                                            ,port_gdf= port_polygons
                                            # ,port_name= "Chennai" # commented to get all ports
                                            ,buffer_radius=1000 #in meters
                                            # ,anonymized_mmsis=["dfcce95e07"] # commented to get all vessels
                                            )

### **Spatial Metrics**:
    
**Objective**
    
To pinpoint high-traffic areas, such as anchorage zones or congested regions, using vessel trajectory data. This helps stakeholders better understand maritime traffic patterns, improve port planning, and optimize anchorage operations.

**Process**:
- Vessel trajectories were aggregated spatially to identify regions with high densities of vessel activity.
- Data points were mapped using **heatmap visualization**, highlighting areas of concentrated activity.

*The cell output is saved at "vessels_heatmap.html"

In [ ]:
importlib.reload(helper_utils_level_3)
helper_utils_level_3.analyze_vessel_activity(
                                    data= raw_data
                                    # ,anonymized_mmsis=["dfcce95e07","deb97e3414"]
                                    # ,map_save_directory="vessels_heatmap.html"
                                    )

## Level 4:  Strategic Insights 

### **Port Congestion Analysis**

**Overview**

The Port Congestion Analysis provides insights into the congestion levels of ports based on waiting times of vessels. By analyzing key metrics such as total waiting time and the number of distinct vessels, the analysis highlights congested ports to inform operational decisions and resource allocation.

**Benefits**

- **Operational Efficiency**: Identifies bottlenecks in port operations for targeted improvements.
- **Resource Allocation**: Assists in deploying resources to congested areas.

**Usage**

- Monitor congestion levels across ports regularly to guide operational strategies.
- Compare congestion metrics over time to identify trends or impacts of implemented changes.

In [ ]:
importlib.reload(helper_utils_level_4)

# port_list = ["Durban", "Nynashamn", "Bangkok", "Ambarli"]
port_list = port_polygons.iloc[:20,:]["port_name"]
all_waiting_summary = pd.DataFrame()

for port in port_list:
    
    waiting_summary, idle_data, m = helper_utils_level_4.analyze_waiting_times(data = raw_data, 
                                                        ports_df = port_polygons, 
                                                        port_name = port, 
                                                        # center_coords, 
                                                        offset= 1, # in degrees, 1 degree ~= 111.111KM
                                                        speed_threshold= 1, 
                                                        waiting_time_threshold= pd.Timedelta(minutes=30), 
                                                        start_time= None, 
                                                        end_time = None, 
                                                        map_save_path = None)
    if waiting_summary is not None:
        waiting_summary["port_name"] = port
        all_waiting_summary = pd.concat([all_waiting_summary, waiting_summary], ignore_index=True)
        # display(m) # Uncomment to see each ports's map 
all_waiting_summary

In [ ]:
importlib.reload(helper_utils_level_4)

port_summary, plot = helper_utils_level_4.analyze_port_congestion(all_waiting_summary,number_of_days_thrsh=5)
plot

In [ ]:
port_summary

### **Time Series Forecasting for Vessel Traffic Using ARIMA**

**Overview**

The ARIMA (AutoRegressive Integrated Moving Average) model predicts future vessel traffic to ports based on historical time series data. This forecasting helps anticipate vessel traffic or trajectory counts, aiding in planning for port resources and avoiding bottlenecks.

**Benefits**

- **Proactive Planning**: Predict future vessel traffic to prepare resources accordingly.
- **Trend Analysis**: Understand traffic patterns over time and anticipate peak periods.

**Usage**

- Use for port traffic forecasting to prevent congestion and optimize scheduling.
- Combine with congestion analysis to project future port congestion levels.

In [ ]:
importlib.reload(helper_utils_level_4)

port_name = "Ambarli"
# port_name = "Durban"

aggregated_data = helper_utils_level_4.prepare_traffic_data(raw_data= raw_data,
                                                        port_gdf= port_polygons,
                                                        port_name= port_name,
                                                        center_coords = None,
                                                        offset = 1.0,
                                                        time_interval = "daily")

result_df, plt = helper_utils_level_4.forecast_traffic_ARIMA(aggregated_data, 
                                                     metric='trajectory_count', 
                                                     forecast_period=7, 
                                                     granularity='daily')

display(aggregated_data)
display(plt)
result_df

In [ ]:
importlib.reload(helper_utils_level_4)

port_name = "Ambarli"
# port_name = "Durban"

aggregated_data = helper_utils_level_4.prepare_traffic_data(raw_data= raw_data,
                                                        port_gdf= port_polygons,
                                                        port_name= port_name,
                                                        center_coords = None,
                                                        offset = 1.0,
                                                        time_interval = "daily")

result_df, plt = helper_utils_level_4.forecast_traffic_ARIMA(aggregated_data, 
                                                     metric='vessel_count', 
                                                     forecast_period=7, 
                                                     granularity='daily')

display(aggregated_data)
display(plt)
result_df

In [ ]:
importlib.reload(helper_utils_level_4)

port_name = "Ambarli"
# port_name = "Durban"

aggregated_data = helper_utils_level_4.prepare_traffic_data(raw_data= raw_data,
                                                        port_gdf= port_polygons,
                                                        port_name= port_name,
                                                        center_coords = None,
                                                        offset = 1.0,
                                                        time_interval = "daily")

result_df, plt = helper_utils_level_4.forecast_traffic_ARIMA(aggregated_data, 
                                                     metric='vessel_count', 
                                                     forecast_period=7, 
                                                     granularity='daily')

display(aggregated_data)
display(plt)
result_df

## **Extra**



### **Main Technical Challenge: Data Size and Computational Complexity**

Handling large maritime datasets posed a significant technical challenge due to the intensive **spatial computations** involved in identifying vessel trajectories and interactions near ports. Below is a comprehensive breakdown of how this challenge was addressed:

---

### **Challenges with Original Geopandas Approach**

- **Computational Overhead**: Geopandas operations, such as calculating buffers and checking point-in-polygon relationships for thousands of trajectories across multiple ports, proved highly **resource-intensive**.
    - **Time Complexity**: Each spatial operation involved iterating over vast datasets, leading to processing times of **1-2 hours** or more.
    - **Memory Usage**: The geospatial operations consumed substantial memory, which could lead to performance bottlenecks.

---

### **Optimized Solution Using Pandas**

To overcome these challenges, I implemented a **bounding-box filtering approach** using Pandas. This drastically reduced computational overhead while maintaining accuracy.

### **Steps Taken**

1. **Define a Bounding Box**:
    - Instead of using a **spatial buffer**, I created a **bounding box** around each port’s center coordinates (latitude and longitude).
    - The bounding box was defined by an offset of ±1 degree, corresponding to a geographical range of ~111 km (1 degree ≈ 111 km).
2. **Filter Data Using Pandas**:
    - Applied **simple column filtering** to include only records within the bounding box.
    - Eliminated the need for expensive point-in-polygon checks.

In [ ]:
importlib.reload(helper_utils)
summary, map = helper_utils.analyze_trajectories_in_region(
                                                    data= raw_data
                                                    ,ports_df= port_polygons 
                                                    # ,center_coords= (-31,46)
                                                    ,port_name="Chennai"
                                                    ,offset=1 # 1 degree ~111.11KM * 2
                                                    ,start_time=None
                                                    ,end_time=None
                                                    )
print(summary)
map

In [ ]:
importlib.reload(helper_utils)
summary, map = helper_utils.analyze_trajectories_in_region(
                                                    data= raw_data
                                                    ,ports_df= port_polygons 
                                                    # ,center_coords= (-31,46)
                                                    ,port_name="Beira"
                                                    ,offset=1 # 1 degree ~111.11KM * 2
                                                    # ,start_time=None
                                                    # ,end_time=None
                                                    )
print(summary)
map